<a href="https://colab.research.google.com/github/buun1030/matching_engine/blob/main/matching_engine_usecase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# เชื่อมต่อ Google Colab กับ Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandasql
import pandas as pd
import pandasql as ps
import numpy as np
import re
from google.colab import files

In [ ]:
xls = pd.ExcelFile('/content/drive/MyDrive/matching_engine_data.xlsx')
case1 = pd.read_excel(xls, 'Case1')
case2 = pd.read_excel(xls, 'Case2')
case3 = pd.read_excel(xls, 'Case3')
case4 = pd.read_excel(xls, 'Case4')
case5 = pd.read_excel(xls, 'Case5')
case6 = pd.read_excel(xls, 'Case6')
case7 = pd.read_excel(xls, 'Case7')
case8 = pd.read_excel(xls, 'Case8')

In [ ]:
df_bid   = pd.DataFrame()
df_offer = pd.DataFrame()

# Choose use case
cx = case7
df_bid   = cx[['Bid','VBid']].sort_values(by=['Bid'], ascending=False).reset_index(drop=True)
df_offer = cx[['Offer','VOffer']].sort_values(by=['Offer'], ascending=True).reset_index(drop=True)

In [ ]:
def matching_engine(df_bid, df_offer):

  # Check whether max(Bid) < min(Offer) case
  if df_bid['Bid'][0] < df_offer['Offer'][0]:
    return 'Dont have MCP'

  # Check whether max(Bid) = min(Offer) case
  if df_bid['Bid'][0] == df_offer['Offer'][0]:
    mcp = df_bid['Bid'][0]
    final_vol = min(df_bid['VBid'][0], df_offer['VOffer'][0])
    return mcp, final_vol

  # Initial variables prevent local var ref error
  mcp = 0
  final_vol = 0

  # Count number of order book
  len_bid   = len(df_bid)
  len_offer = len(df_offer)

  # Total volume
  total_bid_vol   = sum(df_bid['VBid'].replace(np.nan, 0))
  total_offer_vol = sum(df_offer['VOffer'].replace(np.nan, 0))

  # Check whether run out of order case
  if total_bid_vol < total_offer_vol:

    final_vol = total_bid_vol
    # find index
    accum_offer_vol = 0
    idx = -1
    last_idx = 0
    while accum_offer_vol < final_vol:
      idx = idx + 1
      accum_offer_vol = accum_offer_vol + df_offer['VOffer'][idx]
    while ~np.isnan(df_bid['Bid'][last_idx]) and last_idx < len_bid-1:
      last_idx = last_idx + 1
    if np.isnan(df_bid['Bid'][last_idx]):
      last_idx = last_idx - 1
    if df_bid['Bid'][last_idx] >= df_offer['Offer'][idx]:
      mcp = (df_bid['Bid'][last_idx] + df_offer['Offer'][idx]) / 2
      return mcp, final_vol
  else:

    final_vol = total_offer_vol
    # find index
    accum_bid_vol = 0
    idx = -1
    last_idx = 0
    while accum_bid_vol < final_vol:
      idx = idx + 1
      accum_bid_vol = accum_bid_vol + df_bid['VBid'][idx]
    while ~np.isnan(df_offer['Offer'][last_idx]) and last_idx < len_offer-1:
      last_idx = last_idx + 1
    if np.isnan(df_offer['Offer'][last_idx]):
      last_idx = last_idx - 1
    if df_bid['Bid'][idx] >= df_offer['Offer'][last_idx]:
      mcp = (df_bid['Bid'][idx] + df_offer['Offer'][last_idx]) / 2
      return mcp, final_vol 
  
  # Initial accumulated volume
  accum_bid_vol   = 0
  accum_offer_vol_qm2 = 0
  accum_offer_vol_qm1 = 0
  accum_offer_vol_qm0 = 0

  for p in range(0, len_bid):

    # Accumulate bid volume
    accum_bid_vol   = accum_bid_vol + df_bid['VBid'][p]

    # Reset offer volume each time bid price has changed
    accum_offer_vol = 0

    for q in range(0, len_offer):

      # Accumulate offer volume
      if q == 0:
        accum_offer_vol_qm2 = df_offer['VOffer'][0]
        accum_offer_vol_qm1 = df_offer['VOffer'][0]
        accum_offer_vol_qm0 = df_offer['VOffer'][0]
      elif q == 1:
        accum_offer_vol_qm2 = df_offer['VOffer'][0]
        accum_offer_vol_qm1 = df_offer['VOffer'][0]
        accum_offer_vol_qm0 = df_offer['VOffer'][0] + df_offer['VOffer'][1]
      else:
        accum_offer_vol_qm2 = accum_offer_vol_qm1
        accum_offer_vol_qm1 = accum_offer_vol_qm0
        accum_offer_vol_qm0 = accum_offer_vol_qm0 + df_offer['VOffer'][q]
      
      # Actually for dedugging
      print('Bid_p = ' + str(df_bid['Bid'][p]))
      print('Offer_q = ' + str(df_offer['Offer'][q]))
      print('accum_bid_vol = ' + str(accum_bid_vol))
      print('accum_offer_vol_qm1 = ' + str(accum_offer_vol_qm1))
      print('accum_offer_vol_qm2 = ' + str(accum_offer_vol_qm2))
      print('---------------------------------------')

      # To avoid error
      if q < 2:
        qmi1 = 0
        qmi2 = 0
      else:
        qmi1 = q-1
        qmi2 = q-2

      # Check wheather bid price already cross offer price
      if df_bid['Bid'][p] <= df_offer['Offer'][q] and \
        (df_bid['Bid'][p] >= df_offer['Offer'][qmi1] or q-1 < 0) and \
        (accum_bid_vol >= accum_offer_vol_qm1 or \
        (accum_bid_vol <= accum_offer_vol_qm1 and \
         accum_bid_vol >= accum_offer_vol_qm2)):
         

        # Calculate market clearing price
        if df_bid['Bid'][p] == df_offer['Offer'][q] and accum_bid_vol >= accum_offer_vol_qm0:
          mcp = df_bid['Bid'][p]
          final_vol = accum_offer_vol_qm0
        elif df_bid['Bid'][p] == df_offer['Offer'][q] and accum_bid_vol >= accum_offer_vol_qm1:
          mcp = df_bid['Bid'][p]
          final_vol = accum_bid_vol
        elif accum_bid_vol <= accum_offer_vol_qm1 and accum_bid_vol == accum_offer_vol_qm2:
          mcp = (df_bid['Bid'][p] + df_offer['Offer'][qmi2]) / 2
          final_vol = accum_bid_vol
        else:
          mcp = (df_bid['Bid'][p] + df_offer['Offer'][qmi1]) / 2
          final_vol = min(accum_bid_vol, accum_offer_vol_qm1)

        #bid_zone1_result = matching_result(df_bid, mcp, final_vol)
        #offer_zone1_result = matching_result(df_offer, mcp, final_vol)

        # Show result
        print('Bid_price = ' + str(df_bid['Bid'][p]))
        print('Offer_price = ' + str(df_offer['Offer'][qmi1]))
        print('Bid_vol = ' + str(accum_bid_vol))
        print('Offer_vol = ' + str(accum_offer_vol_qm1))
        print('Final_vol = ' + str(final_vol))
        print('MCP = ' + str(mcp))
        print('-----------------Answer----------------')
        return mcp, final_vol

      elif df_bid['Bid'][p] >= df_offer['Offer'][q] and \
          (df_bid['Bid'][p+1] <= df_offer['Offer'][q] or p+1 > df_bid['Bid'].count()-1) and \
          accum_bid_vol <= accum_offer_vol_qm0 and \
          (accum_bid_vol >= accum_offer_vol_qm1 or q-1 < 0):

        # Calculate market clearing price
        if accum_bid_vol == accum_offer_vol_qm1:
          mcp = (df_bid['Bid'][p] + df_offer['Offer'][qmil1]) / 2
        else:
          mcp = (df_bid['Bid'][p] + df_offer['Offer'][q]) / 2

        final_vol = min(accum_bid_vol, accum_offer_vol_qm0)

        #bid_zone1_result = matching_result(df_bid, mcp, final_vol)
        #offer_zone1_result = matching_result(df_offer, mcp, final_vol)

        # Show result
        print('Bid_price = ' + str(df_bid['Bid'][p]))
        print('Offer_price = ' + str(df_offer['Offer'][q]))
        print('Bid_vol = ' + str(accum_bid_vol))
        print('Offer_vol = ' + str(accum_offer_vol_qm1))
        print('Final_vol = ' + str(final_vol))
        print('MCP = ' + str(mcp))
        print('-----------------Answer----------------')
        return mcp, final_vol

      elif df_bid['Bid'][p] == df_offer['Offer'][q] and \
          (df_bid['Bid'][p+1] < df_offer['Offer'][q] or p+1 > df_bid['Bid'].count()-1):


        # Calculate market clearing price
        if accum_bid_vol > accum_offer_vol_qm0:
          mcp = df_bid['Bid'][p]
          final_vol = accum_offer_vol_qm0
  

In [ ]:
mcp, final_vol = matching_engine(df_bid, df_offer)

Bid_p = 2.51
Offer_q = 2.31
accum_bid_vol = 50
accum_offer_vol_qm1 = 10
accum_offer_vol_qm2 = 10
---------------------------------------
Bid_p = 2.51
Offer_q = 2.51
accum_bid_vol = 50
accum_offer_vol_qm1 = 10
accum_offer_vol_qm2 = 10
---------------------------------------
Bid_price = 2.51
Offer_price = 2.31
Bid_vol = 50
Offer_vol = 10
Final_vol = 30
MCP = 2.51
-----------------Answer----------------


In [ ]:
# Prepare for display in table
def matching_result(df, mcp, final_vol):
  df_final = df
  accum_vol = 0
  over = 0

  for r in range(0, len(df)):

    df_final.iloc[:,0][r] = mcp
    accum_vol = accum_vol + df_final.iloc[:,1][r]

    if accum_vol > final_vol and over == 0:
      df_final.iloc[:,1][r] = final_vol - accum_vol + df.iloc[:,1][r]
      over = 1
    elif over == 1:
      df_final.iloc[:,1][r] = 0
    
  return df_final

In [ ]:
df_offer['Offer'][len(df_offer)-1]

In [ ]:
mcp

In [ ]:
# Old version: Prepare for display in table
df_bid_final = df_bid
df_offer_final = df_offer
accum_bid_vol   = 0
accum_offer_vol = 0

over = 0
for r in range(0, len(df_bid)):

  accum_bid_vol = accum_bid_vol + df_bid['VBid'][r]

  df_bid_final['Bid'][r] = mcp

  if accum_bid_vol > final_vol and over == 0:
    df_bid_final['VBid'][r] = final_vol - accum_bid_vol + df_bid['VBid'][r]
    over = 1
  elif over == 1:
    df_bid_final['VBid'][r] = 0

over = 0
for r in range(0, len(df_offer)):

  accum_offer_vol = accum_offer_vol + df_offer['VOffer'][r]

  df_offer_final['Offer'][r] = mcp

  if accum_offer_vol > final_vol and over == 0:
    df_offer_final['VOffer'][r] = final_vol - accum_offer_vol + df_offer['VOffer'][r]
    over = 1
  elif over == 1:
    df_offer_final['VOffer'][r] = 0